In [1]:
# Install required libraries
!pip install PyPDF2 sentence-transformers chromadb transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 927.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.6/602.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.6 MB/s eta

In [7]:
# Load pdf
import PyPDF2
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]
print(f"Uploaded file: {pdf_path} ")

# Function to validate and get basic information about the PDF
def validate_pdf(pdf_path):
    reader = PyPDF2.PdfReader(pdf_path)
    num_pages = len(reader.pages)

    for page_number in range(num_pages):
        page = reader.pages[page_number]
        text = page.extract_text()
        print(f"Page {page_number + 1}: {len(text)} characters")

validate_pdf(pdf_path)

Saving constitution.pdf to constitution (2).pdf
Uploaded file: constitution (2).pdf 
Page 1: 185 characters
Page 2: 890 characters
Page 3: 1013 characters
Page 4: 1175 characters
Page 5: 1168 characters
Page 6: 1275 characters
Page 7: 1254 characters
Page 8: 1446 characters
Page 9: 1380 characters
Page 10: 1299 characters
Page 11: 1501 characters
Page 12: 1306 characters
Page 13: 1105 characters
Page 14: 1602 characters
Page 15: 1246 characters
Page 16: 1398 characters
Page 17: 1323 characters
Page 18: 1125 characters
Page 19: 1245 characters
Page 20: 1575 characters
Page 21: 1572 characters
Page 22: 1306 characters
Page 23: 1428 characters
Page 24: 1241 characters
Page 25: 1584 characters
Page 26: 1317 characters
Page 27: 1546 characters
Page 28: 1452 characters
Page 29: 1588 characters
Page 30: 1201 characters
Page 31: 1122 characters
Page 32: 848 characters
Page 33: 1440 characters
Page 34: 2138 characters
Page 35: 1973 characters
Page 36: 2162 characters
Page 37: 2221 characters
Pa

In [8]:
# PDF to chunks
import spacy

# Load English language model for sentence segmentation
nlp = spacy.blank('en')
nlp.add_pipe('sentencizer')

# Function to split PDF into sentences and then into chunks
def pdf_to_chunks(pdf_path, chunk_size=10):
    reader = PyPDF2.PdfReader(pdf_path)
    num_pages = len(reader.pages)
    pdf_info = []

    for page_number in range(num_pages):
        page = reader.pages[page_number]
        text = page.extract_text()

        # Split text into sentences
        doc = nlp(text)
        sentences = [sent.text.strip() for sent in doc.sents]

        # Create chunks of sentences
        chunks = [' '.join(sentences[i:i + chunk_size]) for i in range(0, len(sentences), chunk_size)]

        # Store each page's chunks in a dictionary
        pdf_info.append({
            'page': page_number + 1,
            'chunks': chunks
        })

    return pdf_info

# Get chunks from the PDF
chunks_info = pdf_to_chunks(pdf_path)


In [ ]:
# Chunks to Embedding
from sentence_transformers import SentenceTransformer

# Load the embedding model
embedding_model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)

# Convert chunks into embeddings
def create_embeddings(chunks_info):
    all_embeddings = []
    for page_data in chunks_info:
        for chunk in page_data['chunks']:
            embedding = embedding_model.encode(chunk, convert_to_tensor=True)
            all_embeddings.append((page_data['page'], chunk, embedding))
    return all_embeddings

# Create embeddings from the chunks
embeddings = create_embeddings(chunks_info)

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling.py:   0%|          | 0.00/59.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [11]:
# Storing chunks in ChromaDB
import chromadb

# Initialize ChromaDB client
client = chromadb.Client()

# Create a collection to store PDF embeddings
collection = client.create_collection(name="pdf_embeddings")

# Store the embeddings with IDs and chunks
for idx, (page, chunk, embedding) in enumerate(embeddings):
    collection.add(
        documents=[chunk],
        embeddings=[embedding.cpu().numpy()],
        ids=[f"doc_{page}_{idx}"]
    )